In [1]:
import numpy as np
from glob import glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [2]:
images_path = 'Data/' #Путь к данным определяем

In [3]:
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 30
# Размер мини-выборки. Будем брать в каждой эпохе будем рассматривать 64 изображения
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 832
# Количество изображений для проверки
nb_validation_samples = 332
# Количество изображений для тестирования
nb_test_samples = 332

In [4]:
model = Sequential()
#150*150 разбивается на 32 части
# 32 нейрона = 32 признака, 1 нерон выдит 3*3 пикселей
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# преобразует из двумерного в одномерное представленние
model.add(Flatten())


model.add(Dense(64))
model.add(Activation('relu'))

#50 процентов (случайно) нейронов будет выключать в эпохе. Защита от переучивания сети
model.add(Dropout(0.5))

#4 категории
model.add(Dense(4))
model.add(Activation('sigmoid'))

In [5]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
datagen = ImageDataGenerator(rescale=1. / 255) #Делит значения пикселя на 255

In [7]:
train_generator = datagen.flow_from_directory(#генератор для уобучения. Как доставать эти изображения
    images_path+'train/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 556 images belonging to 4 classes.


In [8]:
val_generator = datagen.flow_from_directory(#генератор для ваилидации
    images_path+'val/',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 124 images belonging to 4 classes.


In [9]:
test_generator = datagen.flow_from_directory(#генератор для тестов.
    images_path+'test',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 127 images belonging to 4 classes.


In [10]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
13/13 [==============================] - 37s 3s/step - loss: 1.1199 - accuracy: 0.5354 - val_loss: 0.6056 - val_accuracy: 0.8077
Epoch 2/30
13/13 [==============================] - 32s 2s/step - loss: 0.5489 - accuracy: 0.7672 - val_loss: 0.2504 - val_accuracy: 0.9519
Epoch 3/30
13/13 [==============================] - 34s 3s/step - loss: 0.3161 - accuracy: 0.9150 - val_loss: 0.0702 - val_accuracy: 0.9872
Epoch 4/30
13/13 [==============================] - 33s 3s/step - loss: 0.1652 - accuracy: 0.9557 - val_loss: 0.0204 - val_accuracy: 0.9936
Epoch 5/30
13/13 [==============================] - 32s 2s/step - loss: 0.1199 - accuracy: 0.9715 - val_loss: 0.0191 - val_accuracy: 0.9904
Epoch 6/30
13/13 [==============================] - 33s 3s/step - loss: 0.0844 - accuracy: 0.9844 - val_loss: 0.0085 - val_accuracy: 1.0000
Epoch 7/30
13/13 [==============================] - 33s 3s/step - loss: 0.0637 - accuracy: 0.9823 - val_loss: 0.0040 - val_accuracy: 1.0000
Epoch 8/30
13/13 [==

In [11]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("hack_model_150p_30epochs.json", "w")
json_file.write(model_json)
json_file.close()

In [12]:
model.save_weights("hack_model_150p_30epochs.h5")

In [13]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 98.43%


In [15]:
for i in glob('*.png'):
    test_image = image.load_img(i, target_size = (150, 150))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    print(result)
    #acc = model.evaluate(val_generator, verbose=0)
    #print(acc)
    if result[0][0] == 1:
        prediction = 'asterisk'
    elif result[0][1] == 1:
        prediction = 'blade'
    elif result[0][2] == 1:
        prediction = 'gun'
    elif result[0][3] == 1:
        prediction ='knife'
    print('prediction', prediction)